In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [1]:
import boto3
from bs4 import BeautifulSoup
import requests
import joblib
from tqdm import tqdm
import pandas as pd

c:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
#関数１
def joblib_get_url(i):
    url_list_pre=list()
    url=url_list_001[i]
    raw_url="https://www.baitoru.com"
    res=requests.get(url)
    soup=BeautifulSoup(res.text,"html.parser")
    for i in range(0,20):
        try:
            elem = soup.find_all("article", class_="list-jobListDetail")[i].find("h3").find("a")
            url_key=elem.attrs["href"] #hrefの値(url)のみを抜き取る
            n_url_key=raw_url+url_key
            url_list_pre.append(n_url_key)
        except:
            pass
        
    return url_list_pre

In [3]:
#関数2
def joblib_get_data(i):
    new_list=list()
    houjin=syokusyu=kyuyo=''
    
    url = url_list_003[i]
    res = requests.get(url)
    soup=BeautifulSoup(res.text,"html.parser")
    
    try:
        selector="#contents > div.layout-grid-2-2 > div.layout-column-1 > div > div.detail-companyInfo > div > div.bg01 > div.pt02 > dl > dd > p"
        raw_houjin=soup.select_one(selector).get_text()
        houjin=raw_houjin.split()
    except:
        pass
    
    
    for i in range(0, 10):
        try:
            judge = soup.find("div", class_="detail-basicInfo").find_all("dt")[i].get_text()
            if "職種" in judge:
                pre_syokusyu = soup.find("div", class_="detail-basicInfo").find_all("dd")[i].get_text()
                syokusyu = pre_syokusyu.split()
        except:
            pass

    for i in range(0, 10):
        try:
            judge = soup.find("div", class_="detail-basicInfo").find_all("dt")[i].get_text()
            if "給与" in judge:
                pre_kyuyo = soup.find("div", class_="detail-basicInfo").find_all("dd")[i].get_text()
                kyuyo = pre_kyuyo.split()
        except:
            pass  
    
    new_list.append(houjin)
    new_list.append(syokusyu)
    new_list.append(kyuyo)

    return new_list

In [4]:
url="https://www.baitoru.com/kanto/jlist/tokyo/conveni/"
res=requests.get(url)
soup=BeautifulSoup(res.text,"html.parser")
total_num_str_pre=soup.find(id="js-job-count").get_text()

print(total_num_str_pre)

2,173


In [5]:
total_num_str=total_num_str_pre.replace(",","")
total_num=int(total_num_str)
print(total_num)

2173


In [6]:
total_page_num=total_num//20+1
print(total_page_num)

109


In [7]:
url_list_001=list()
url_list_001.append(url)

In [8]:
a=10
joblib_num=total_page_num//a+1

In [15]:
url_list_002=list()
for n in tqdm(range(0,joblib_num)):
    try:
        resultList = joblib.Parallel(n_jobs=12, verbose=3)( [joblib.delayed(joblib_get_url)(i) for i in range(n*a,(n+1)*a)])
        url_list_002.extend(resultList)
    except:
        pass

  0%|          | 0/1 [00:00<?, ?it/s][Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   3 out of  10 | elapsed:    2.0s remaining:    4.7s
[Parallel(n_jobs=12)]: Done   7 out of  10 | elapsed:    2.2s remaining:    0.9s
100%|██████████| 1/1 [00:08<00:00,  8.28s/it]


In [16]:
url_list_002_filtered=[x for x in url_list_002 if x is not None]
flatten_url_list_002 = [ flatten for inner in url_list_002_filtered for flatten in inner ]#難しい(ノンローカル変数)
url_list_003 = []
for i in flatten_url_list_002:
    if i not in url_list_003:
        url_list_003.append(i)

In [17]:
b=100
joblib_num = len(url_list_003)//b + 1

all_list = list()

In [18]:
for n in tqdm(range(0, joblib_num)):
    try:
        resultList = joblib.Parallel(n_jobs=12, verbose=3)( [joblib.delayed(joblib_get_data)(i) for i in range(n*b,(n+1)*b)])
        all_list.extend(resultList)
    except:
        pass

100%|██████████| 1/1 [00:02<00:00,  2.26s/it]


In [19]:
#flatten_list = [ flatten for inner in all_list for flatten in inner ]
all_list_filtered = [x for x in all_list if x is not None]
kekka = pd.DataFrame(all_list_filtered,columns=["法人名","職種","給与情報"])


kekka.to_csv('second.csv',encoding="utf-8-sig")

In [14]:
print('データの格納')
bucket_name = 'monthly-scraping'
s3_key = 'monthly_data/{}年/{}月/媒体名.csv'.format(year,month)
s3 = boto3.resource('s3') 

s3_obj = s3.Object(bucket_name,s3_key)
s3_obj.put(Body=kekka.to_csv(None).encode('utf_8_sig'))

データの格納


NameError: name 'year' is not defined